In [1]:
import json
import random
import re
import os
import pandas as pd
from tqdm.notebook import tqdm
from transformers import pipeline
from typing import Dict
import dotenv
dotenv.load_dotenv()

/Users/jenitza/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


True

In [7]:
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
!huggingface-cli login --token HUGGINGFACE_API_KEY

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Traceback (most recent call last):
  File "/Users/jenitza/anaconda3/bin/huggingface-cli", line 11, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/jenitza/anaconda3/lib/python3.11/site-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/Users/jenitza/anaconda3/lib/python3.11/site-packages/huggingface_hub/commands/user.py", line 101, in run
    login(token=self.args.token, add_to_git_credential=self.args.add_to_git_credential)
  File "/Users/jenitza/anaconda3/lib/python3.11/site-packages/huggingface_hub/_login.py", line 96, in login
    _login(token, add_to_git_credential=add_to_git_credential, write_permission=write_permission)
  File "/Users/jenitza/anaconda3/lib/python3.11/site-packages/huggingface_hub/_login.py", line 275, in _login
    raise ValueError("Invalid token passed!")
ValueErr

In [3]:
df = pd.read_csv("datasets/cleaned_dataset.csv", encoding="utf-8")
df.head()

,Question,Answer
0,How do I apply for CPT?,You must get approval from your Designated Sch...
1,What documents are required for CPT?,Typically you need:\n- CPT Request Form from y...
2,How long does CPT processing take?,Processing usually takes 5-10 business days af...
3,Can I work before receiving CPT authorization?,No you must wait for approval and receive an u...
4,Do I need to pay any fees for CPT?,No CPT does not require a separate application...


In [4]:
LETTER_TO_INDEX = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

class Example:
    def __init__(self, question: str, choice1: str, choice2: str, choice3: str, choice4: str):
        self.question = question
        self.choice1 = choice1
        self.choice2 = choice2
        self.choice3 = choice3
        self.choice4 = choice4

    @property
    def choices(self):
        return [self.choice1, self.choice2, self.choice3, self.choice4]

def _base_prompt() -> str:
    return """Act as an expert legal assistant with comprehensive knowledge of statutory law and case precedent. Analyze the following legal question carefully, then select the correct answer from the given options through rigorous legal reasoning."""

def _format_choices(options: list[str]) -> str:
    return "\n".join(f"({chr(65 + i)}) {choice}" for i, choice in enumerate(options))

def _build_question_section(example: Example) -> str:
    return f"\n\nQuestion: {example.question}\nChoices:\n{_format_choices(example.choices)}"

def _build_instructions() -> str:
    return """\n\n
        Instructions:
        1. Conduct thorough legal analysis of all options
        2. Consider relevant statutes, regulations, and judicial interpretations
        3. Identify potential ambiguities or counterarguments
        4. Select only the BEST supported answer
        5. Respond SOLELY with the correct letter (A-D)

        Answer using this format:
        [X]"""

def _build_final_instruction() -> str:
    return "\n\nPlease reply only with the correct option, do not say anything else.\""

def _prepare_examples(example: Example, no: int = 5) -> str:
    filtered_df = df[df['Question'] != example.question].sample(frac=1)
    examples = []
    
    for _, row in filtered_df.head(no).iterrows():
        right_answer = str(row['Answer'])
        option = [right_answer]
        
        distractors = df[df['Answer'] != right_answer]['Answer'].astype(str).unique()
        if len(distractors) < 3:
            raise ValueError("Not enough unique distractors in the DataFrame.")
        option += random.sample(list(distractors), 3)
        
        random.shuffle(option)
        correct_letter = chr(option.index(right_answer) + 65)
        
        example_str = (
            f"\n\nQuestion: {row['Question']}"
            f"\nChoices:\n{_format_choices(option)}"
            f"\nThe correct answer is ({correct_letter})"
        )
        examples.append(example_str)
    
    return "--- START OF EXAMPLES ---\n" + "".join(examples) + "\n\n--- END OF EXAMPLES ---\n"

def chain_of_thought_prompt(example: Example, max_new_tokens: int = 256) -> str:
    prompt = _base_prompt() + _build_question_section(example)
    prompt += f"\n\nLet's analyze this step by step. First, understand the question. Next, evaluate each option in short (2-5) lines each. Also, you can generate up to {max_new_tokens} tokens to reason."

    cot_output = pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        pad_token_id=pipe.tokenizer.eos_token_id,
        num_beams=5,
        early_stopping=True,
        eos_token_id=pipe.tokenizer.eos_token_id,
        temperature=0.5
    )
    cot_reasoning = cot_output[0]['generated_text']
    prompt = f"{cot_reasoning}\n\nBased on the above, reasoning what is the single, most likely answer choice? \nAnswer using this format:\n[X]"
    prompt += _build_final_instruction()
    return prompt

def five_shot_prompt(example: Example, no: int = 5) -> str:
    prompt = _base_prompt()
    prompt += "\nHere are some example questions from experts. Answer the final question yourself, following the format of the previous questions exactly.\n"
    prompt += _prepare_examples(example=example, no=no)
    prompt += "\n\nNow your turn. Choose the correct option that answers the below question.\n"
    prompt += _build_question_section(example)
    prompt += _build_instructions()
    prompt += _build_final_instruction()
    return prompt

def zero_shot_prompt(example: Example) -> str:
    prompt = _base_prompt() + _build_question_section(example)
    prompt += _build_instructions()
    prompt += _build_final_instruction()
    return prompt

In [5]:
# # delete empty rows
# df = df.dropna()

In [6]:
# # remove ',' from df[Answer]
# df['Answer'] = df['Answer'].str.replace(',', '')

In [7]:
# df.to_csv("../dataset/cleaned_dataset.csv", index=False, encoding="utf-8")
model_name =  "meta-llama/Llama-3.1-8B-Instruct"# "meta-llama/Llama-3.1-8B-Instruct"
pipe = pipeline ("text-generation", model=model_name, device='cuda')

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda


In [8]:
# Initialize the model pipeline
#pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct", device='cuda')

def runner(n=1, prompt_type='zero_shot_prompt'):
   # model_name = "meta-llama/Llama-3.2-1B-Instruct"
    for i in tqdm(range(n), desc="Iterations"):
        results = []
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
            try:
                question = row['Question']
                right_answer = row['Answer']
                # selecting distractions.
                option = []
                option.append(right_answer)
                while len(option) < 4:
                    distractor = df.sample(1)['Answer'].values[0]
                    if distractor not in option and distractor != right_answer:
                        option.append(distractor)

                # print(f"Question: {question}")
                # print(f"Right Answer: {right_answer}")
                # print(f"Options: {option}")
                # print(len(option)) 
                # break

                # Create an example
                random.shuffle(option)
                # Right option is 
                right_option_index = option.index(right_answer)
                right_option_letter = chr(ord('A') + right_option_index)
                #print(f"\n\nRight option index: {right_option_letter}")
                
                example = Example(question, 
                                  option[0], 
                                  option[1], 
                                  option[2], 
                                  option[3]
                                )
                
                # Depending on prompt_type, generate the prompt using the integrated functions
                if prompt_type == 'zero_shot_prompt':
                    prompt = zero_shot_prompt(example)
                elif prompt_type == 'five_shot_prompt':
                    n_examples = 5
                    prompt =  five_shot_prompt(example, n_examples)
                elif prompt_type == 'chain_of_thought_prompt':
                    thinking_tokens = 512
                    prompt = chain_of_thought_prompt(example, thinking_tokens)
                else:
                    # Default behavior (original prompt)
                    print("SELECTING DEFAULT PROMPTING TECHNIQUE")
                    prompt = zero_shot_prompt(example)

                #print(prompt)
                #break
                # Generate the model's response
                response = pipe(
                    prompt,
                    max_new_tokens=15,
                    pad_token_id=pipe.tokenizer.eos_token_id,
                    num_beams=5,
                    early_stopping=True,
                    eos_token_id=pipe.tokenizer.eos_token_id,
                    temperature=0.1
                )

                # Extract generated text
                generated_text = response[0]["generated_text"]
                #print(generated_text)
                answer_portion = generated_text.split("else.")[1]
                match = re.search(r'[A-D]', answer_portion)
                if match:
                    model_answer_letter = match.group()
                else:
                    model_answer_letter = 'NA'
                
                                
                if index == 0 and i==0:
                    print(prompt)
                    print("Correct Ans: ", right_answer)
                    print("Model Replied with: ", model_answer_letter)
                    
                is_correct = (right_option_letter and model_answer_letter) and right_option_letter.lower() == model_answer_letter.lower()

                result = {
                    "iteration": i,
                    "id": index,
                    "model": model_name,
                    "prompt_type": prompt_type,
                    "question": question,
                    "right_answer": right_answer,
                    "right_answer_option": right_option_letter,
                    "model_answer_letter": model_answer_letter,
                    "generated_text": generated_text,
                    "is_correct": str(is_correct)
                }
                results.append(result)

            except Exception as e:
                print(f"Error processing row {index}: {e}")
                print(generated_text)
                continue 

        # Save the results
        if not os.path.exists(f"results/{prompt_type}/{model_name.split('/')[1]}"):
            os.makedirs(f"results/{prompt_type}/{model_name.split('/')[1]}")

        with open(f"results/{prompt_type}/{model_name.split('/')[1]}/output_{i}.json", "w") as f:
                json.dump(results, f, indent=4)
    
        print("Evaluation complete. Results saved to output.json.")    
               

In [ ]:
runner(5, 'chain_of_thought_prompt')

Iterations:   0%|          | 0/5 [00:00<?, ?it/s]

Processing rows:   0%|          | 0/131 [00:00<?, ?it/s]

Act as an expert legal assistant with comprehensive knowledge of statutory law and case precedent. Analyze the following legal question carefully, then select the correct answer from the given options through rigorous legal reasoning.

Question: How do I apply for CPT?
Choices:
(A) To file Form I-765 online:

 Create a USCIS online account if you don't already have one.

 Select 'File a Form Online.'

 Complete all sections of the form upload required evidence and pay the applicable fee.
(B) You must get approval from your Designated School Official (DSO) and submit a CPT request form along with an offer letter from your employer.
(C) Yes your employer can file an H-1B petition for you while you are on OPT or STEM OPT. If approved you may qualify for Cap-Gap Extension allowing you to work until H-1B status begins.
(D) Generally CPT and OPT require U.S.-based employment. Remote work may be allowed if the employer has a U.S. presence and follows employment regulations.

Let's analyze thi

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [6]:
import json
import glob
import os

prompt_type = 'five_shot_prompt' #"zero_shot_prompt" # 'five_shot_prompt', 'chain_of_thought_prompt'
#model_name = 'meta-llama/Llama-3.2-3B-Instruct'
model_name = "deepseek-ai/deepseek-llm-7b-chat" 
#model_name =  "meta-llama/Llama-3.1-8B-Instruct" # "meta-llama/Llama-3.1-8B-Instruct"

# Construct the directory path
directory = f"../results/{prompt_type}/{model_name.split('/')[1]}/"

# Find all JSON files matching output_*.json pattern
json_files = glob.glob(os.path.join(directory, "output_*.json"))

# Process each JSON file
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Count occurrences of is_correct = True and is_correct = False
    true_count = sum(1 for item in data if item.get("is_correct") == 'True')
    false_count = sum(1 for item in data if item.get("is_correct") == 'False')
                   
    print(f"File: {file_path}")
    print(f"Total is_correct = True: {true_count}")
    print(f"Total is_correct = False: {false_count}")
    print("-" * 40) 


File: ../results/five_shot_prompt/deepseek-llm-7b-chat/output_1.json
Total is_correct = True: 73
Total is_correct = False: 58
----------------------------------------
File: ../results/five_shot_prompt/deepseek-llm-7b-chat/output_0.json
Total is_correct = True: 67
Total is_correct = False: 64
----------------------------------------
File: ../results/five_shot_prompt/deepseek-llm-7b-chat/output_4.json
Total is_correct = True: 68
Total is_correct = False: 63
----------------------------------------
File: ../results/five_shot_prompt/deepseek-llm-7b-chat/output_3.json
Total is_correct = True: 74
Total is_correct = False: 57
----------------------------------------
File: ../results/five_shot_prompt/deepseek-llm-7b-chat/output_2.json
Total is_correct = True: 67
Total is_correct = False: 64
----------------------------------------
